___
# Ciência dos Dados - PROJETO 2

___
## Beni Stern

## Leonardo Costa

## Gabriel Duarte

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Duarte\Downloads\INSPER aqui tbm\DADOS DP\Projeto2\Projeto-2-Filtro-Spam


In [3]:
dados = pd.read_excel("spamham2019.xlsx")

In [4]:
dados_aleatorio = dados.sample(frac=1)
dados_aleatorio = dados.sample(frac=1).reset_index(drop=True)
dados_treinamento = dados_aleatorio.loc[:4180:,:]
dados_teste = dados_aleatorio.loc[4180:,:]

In [5]:
dados_treinamento.head()

,Email,Class
0,Auction round 4. The highest bid is now å£54. ...,spam
1,We not leaving yet. Ok lor then we go elsewher...,ham
2,Ok...,ham
3,We are okay. Going to sleep now. Later,ham
4,Do you still have the grinder?,ham


In [6]:
dados_teste.head()

,Email,Class
4180,Call to the number which is available in appoi...,ham
4181,Ok... Sweet dreams...,ham
4182,Tomarrow i want to got to court. At &lt;DECIM...,ham
4183,"Don't Think About \What u Have Got\"" Think Abo...",ham
4184,Good FRIENDS CaRE for each Other.. CLoSE Frien...,ham


___
## 3. Classificador Naive-Bayes

In [18]:
def Filtro(base):
    lista = []
    for e in base["Email"]:
        #lista_separada = e.split(" ")
        e.replace(",","")
        e.replace("/n", "")
        e.replace('""', "")
        e.replace("'", "")
        e.replace(":", "")
        e.replace(".", "")
        e.replace("...", "")
        e.replace(")", "")
        e.replace("(", "")
        e.replace("", "")

        lista.append(e)
    base2 = pd.DataFrame({"Email":lista,"Class":base["Class"]})
    
    return base2

Filtro(dados).head()

,Class,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
n_ham = dados_treinamento["Class"].value_counts()[0]
n_spam = dados_treinamento["Class"].value_counts()[1]
total = n_spam + n_ham

P_ham = n_ham/total
P_spam = n_spam/total

print("P(SPAM) = ", P_spam)
print("P(HAM)  = ", P_ham)

P(SPAM) =  0.13178665391054772
P(HAM)  =  0.8682133460894523


In [17]:
def probis(date,nome):
    frase = date["Email"]
    palavras_em_lista = []
    
    for linha in frase:
        lista_palavras = linha.split(" ")
        
        for p in lista_palavras:
            palavras_em_lista.append(p)        
    
    total_palavras = len(palavras_em_lista)
    
    palavras = pd.DataFrame(palavras_em_lista)
    contagem = palavras[0].value_counts()
    tabela_contagem = pd.DataFrame(contagem)
    nome_tabela = 'P(word|{})'.format(nome)
    final = tabela_contagem.rename(index=str,columns={0: "Quantidade"})
    
    final[nome_tabela] = final["Quantidade"]/total_palavras
    
    return final

P_wordSpam = probis(dados_treinamento[dados_treinamento.Class == "spam"],"Spam")
P_wordHam = probis(dados_treinamento[dados_treinamento.Class == "ham"],"Ham")

P_wordSpam.head()

,Quantidade,P(word|Spam)
to,435,0.032912
a,268,0.020277
call,137,0.010365
2,136,0.010290
or,134,0.010138


___
## 4. Qualidade do Classificador alterando a base de treinamento